In [1]:
# access the postgresql db

import os
import psycopg2
from dotenv import load_dotenv

# Load environment variables
load_dotenv("../.env.local")

# Get database connection details from environment variables
DATABASE_URL = os.getenv("POSTGRES_URL")
# Get database connection details from URL
# Parse database URL into components
from urllib.parse import urlparse

url = urlparse(DATABASE_URL)
DB_USER = url.username 
DB_PASSWORD = url.password 
DB_HOST = url.hostname 
DB_PORT = url.port 
DB_NAME = url.path[1:] 

# Establish connection
def connect_to_db():
    try:
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
        print("Successfully connected to PostgreSQL database")
        
        # Create a cursor
        cur = conn.cursor()
        
    except psycopg2.Error as e:
        print(f"Error connecting to PostgreSQL database: {e}")
    
    return conn, cur


In [2]:
conn, cur = connect_to_db()
try:
    # Execute query to get the most recent chat
    cur.execute("""
            SELECT * FROM public."Chat"
            ORDER BY "createdAt" DESC 
            LIMIT 1
        """)
        
    # Fetch the result
    last_chat = cur.fetchone()
    
    if last_chat:
        print("Last chat found:")
        print(last_chat)
    else:
        print("No chats found in database")
    
except psycopg2.Error as e:
    print(f"Error querying database: {e}")
    # Rollback the transaction to reset the connection state
    conn.rollback()
    # Close cursor and connection to clean up
    cur.close()
    conn.close()

Successfully connected to PostgreSQL database
Last chat found:
('3e84b92a-6390-438c-ba44-e008081d0b08', datetime.datetime(2025, 4, 30, 14, 22, 34, 395000), '235049eb-15ee-4a2a-9d7f-6189e9bd956e', 'Dijkstras Algorithm Code Example', 'private', 'google-gemini-2.0-flash-exp-image-generation')


In [3]:
import json
conn, cur = connect_to_db()
chat_id = last_chat[0]
# hat_id = '605559b5-9721-46e6-b052-a3084543110d'
try:
    # Execute query to get messages for the last chat
    cur.execute("""
        SELECT * FROM public."Message_v2"
        WHERE "chatId" = %s
        ORDER BY "createdAt" ASC
    """, (chat_id,))  # Use the chat ID from last_chat tuple
    
    # Fetch all messages
    chat_messages = cur.fetchall()
    
    if chat_messages:
        print("Messages for this chat:")
        for message in chat_messages:
            print(json.dumps({
                "id": message[0],
                "chatId": message[1], 
                "role": message[2],
                "parts": message[3],
                "attachments": message[4],
                "createdAt": message[5].isoformat()                
            }, indent=2))
    else:
        print("No messages found for this chat")

except psycopg2.Error as e:
    print(f"Error querying messages: {e}")
    conn.rollback()
finally:
    # Clean up database resources
    cur.close()
    conn.close()


Successfully connected to PostgreSQL database
Messages for this chat:
{
  "id": "488ba53b-798e-4068-a236-9bfcc5c41e5e",
  "chatId": "3e84b92a-6390-438c-ba44-e008081d0b08",
  "role": "user",
  "parts": [
    {
      "text": "Write code to demonstrate djikstra's algorithm",
      "type": "text"
    }
  ],
  "attachments": [],
  "createdAt": "2025-04-30T14:22:34.407000"
}
{
  "id": "b578c1ce-d99e-438e-878d-e781ec29bec7",
  "chatId": "3e84b92a-6390-438c-ba44-e008081d0b08",
  "role": "assistant",
  "parts": [
    {
      "type": "tool-invocation",
      "toolInvocation": {
        "state": "result",
        "toolCallId": "generated_0",
        "toolName": "createDocument",
        "args": {
          "title": "Dijkstra's Algorithm in Python",
          "kind": "code"
        },
        "result": {
          "id": "8811ac4b-07da-468b-b09d-20c22b717b34",
          "title": "Dijkstra's Algorithm in Python",
          "kind": "code",
          "content": "A document (id: 8811ac4b-07da-468b-b09d

In [4]:
import json
conn, cur = connect_to_db()
chat_id = last_chat[0]
chat_id = '605559b5-9721-46e6-b052-a3084543110d'
try:
    # Execute query to get messages for the last chat
    cur.execute("""
        SELECT * FROM public."Message_v2"
        WHERE "chatId" = %s
        ORDER BY "createdAt" ASC
    """, (chat_id,))  # Use the chat ID from last_chat tuple
    
    # Fetch all messages
    chat_messages = cur.fetchall()
    
    if chat_messages:
        print("Messages for this chat:")
        for message in chat_messages:
            print(json.dumps({
                "id": message[0],
                "chatId": message[1], 
                "role": message[2],
                "parts": message[3],
                "attachments": message[4],
                "createdAt": message[5].isoformat()                
            }, indent=2))
    else:
        print("No messages found for this chat")

except psycopg2.Error as e:
    print(f"Error querying messages: {e}")
    conn.rollback()
finally:
    # Clean up database resources
    cur.close()
    conn.close()


Successfully connected to PostgreSQL database
Messages for this chat:
{
  "id": "b2eaa056-2fcd-481a-87f3-d88194ddf0d9",
  "chatId": "605559b5-9721-46e6-b052-a3084543110d",
  "role": "user",
  "parts": [
    {
      "text": "Write code to demonstrate djikstra's algorithm",
      "type": "text"
    }
  ],
  "attachments": [],
  "createdAt": "2025-04-30T08:39:46.605000"
}
{
  "id": "afa59c46-e2de-4ba2-abd6-c279d117f805",
  "chatId": "605559b5-9721-46e6-b052-a3084543110d",
  "role": "assistant",
  "parts": [
    {
      "type": "step-start"
    },
    {
      "type": "text",
      "text": "OK. I can help with that."
    },
    {
      "type": "tool-invocation",
      "toolInvocation": {
        "state": "result",
        "step": 0,
        "toolCallId": "generated_0",
        "toolName": "createDocument",
        "args": {
          "title": "Dijkstra Algorithm in Python",
          "kind": "code"
        },
        "result": {
          "id": "ad671307-2c08-40fb-b183-9d7dafac8cc9",
      

In [5]:
# Connect to DB
conn, cur = connect_to_db()

try:
    # Execute query to get the latest document
    cur.execute("""
        SELECT * FROM public."Document"
        ORDER BY "createdAt" DESC
        LIMIT 1
    """)
    
    # Fetch the document
    document = cur.fetchone()
    
    if document:
        print("Latest document:")
        print(json.dumps({
            "id": document[0],
            "createdAt": document[1].isoformat(),
            "title": document[2], 
            "content": document[3],
            "userId": document[4],
            "text": document[5],
            "chatId": document[6],
        }, indent=2))
    else:
        print("No documents found")

except psycopg2.Error as e:
    print(f"Error querying document: {e}")
    conn.rollback()
finally:
    # Clean up database resources
    cur.close()
    conn.close()


Successfully connected to PostgreSQL database
Latest document:
{
  "id": "8811ac4b-07da-468b-b09d-20c22b717b34",
  "createdAt": "2025-04-30T14:22:39.128000",
  "title": "Dijkstra's Algorithm in Python",
  "content": "import heapq\n\ndef dijkstra(graph, start):\n    distances = {node: float(\"inf\") for node in graph}\n    distances[start] = 0\n    priority_queue = [(0, start)]\n\n    while priority_queue:\n        current_distance, current_node = heapq.heappop(priority_queue)\n\n        if current_distance > distances[current_node]:\n            continue\n\n        for neighbor, weight in graph[current_node].items():\n            distance = current_distance + weight\n            if distance < distances[neighbor]:\n                distances[neighbor] = distance\n                heapq.heappush(priority_queue, (distance, neighbor))\n\n    return distances\n\n\ngraph = {\n    \"A\": {\"B\": 5, \"C\": 2},\n    \"B\": {\"A\": 5, \"D\": 1, \"E\": 4},\n    \"C\": {\"A\": 2, \"F\": 9},\n    \"D